In [105]:
import numpy as np
import pandas as pd

In [ ]:
hsa_hrr = pd.read_excel('./ZipHsaHrr14.xls')

In [146]:
scripts = pd.read_csv('./PARTD_PRESCRIBER_PUF_NPI_DRUG_13.tab', sep='\t')

In [19]:
opioids = scripts.GENERIC_NAME.str.startswith('HYDROCODONE') | scripts.GENERIC_NAME.str.startswith('OXYCODONE') | scripts.GENERIC_NAME.str.startswith('MORPHINE')

In [26]:
neuro = scripts.SPECIALTY_DESC.str.lower().str.contains('neurology')

In [69]:
subset = scripts[neuro & opioids]

In [37]:
subset.NPPES_PROVIDER_CITY

16490                SAGINAW
23861             WELLINGTON
23878             WELLINGTON
23887             WELLINGTON
23888             WELLINGTON
23895             WELLINGTON
23896             WELLINGTON
35275          EAST SYRACUSE
52851             BURNSVILLE
52858             BURNSVILLE
56781              ROSEVILLE
64810           INDIANAPOLIS
74010           LAKE CHARLES
74016           LAKE CHARLES
74073                HAYWARD
80007             GREENVILLE
80042             GREENVILLE
82183           MERRILLVILLE
82311                ARDMORE
86754                 OWOSSO
86796                 OWOSSO
90672              AGUADILLA
94784                HOUSTON
94949                OAKLAND
94972            SAINT LOUIS
94976            SAINT LOUIS
94979            SAINT LOUIS
94980            SAINT LOUIS
99304            CHATTANOOGA
103221          NORTH EASTON
                  ...       
23571422          CARROLLTON
23571426          CARROLLTON
23572453              TACOMA
23573424      

In [42]:
city2zip = {}
for r in hsa_hrr[['zipcode14', 'hrrcity']].iterrows():
    if r[1][1].lower() not in city2zip:
        city2zip[r[1][1].lower()] = r[1][0]

In [57]:
scripts['zipcode'] = scripts.zipcode

In [60]:
scripts.zipcode.isnull().sum()/len(scripts)

0.64213670566228565

In [81]:
foo = hsa_hrr[['hsanum', 'hsacity', 'hsastate', 'hrrnum', 'hrrcity', 'hrrstate']].copy()

In [84]:
bar=foo.drop_duplicates()

In [99]:
ix=bar.loc[0,:].index

In [147]:
for nam in ix: scripts[nam] = np.nan

In [118]:
hsa2row = {}
hrr2row = {}
for r in bar.iterrows():
    if r[1][1].lower() not in hsa2row:
        hsa2row[r[1][1].lower()] = r[0]
    if r[1][4].lower() not in hrr2row:
        hrr2row[r[1][4].lower()] = r[0]

In [163]:
def loc(city):
    city = city.lower()
    if city in hsa2row:
        return hsa_hrr.loc[hsa2row[city]]
    if city in hrr2row:
        return hsa_hrr.loc[hrr2row[city]]
    return None

In [150]:
for r in scripts.index[100:]:
    x = loc(scripts.loc[r, 'NPPES_PROVIDER_CITY'])
    scripts.loc[r, ['hsanum', 'hsacity', 'hsastate', 'hrrcity', 'hrrstate']] = x

KeyboardInterrupt: 

In [152]:
subset = scripts[opioids & neuro]

In [164]:
for r in subset.index:
    x = loc(subset.loc[r, 'NPPES_PROVIDER_CITY'])
    if x is not None:
        subset.loc[r, 'hrrnum'] = x['hrrnum']

/usr/local/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [159]:
subset.head()

,NPI,NPPES_PROVIDER_LAST_ORG_NAME,NPPES_PROVIDER_FIRST_NAME,NPPES_PROVIDER_CITY,NPPES_PROVIDER_STATE,SPECIALTY_DESC,DESCRIPTION_FLAG,DRUG_NAME,GENERIC_NAME,BENE_COUNT,...,TOTAL_CLAIM_COUNT_GE65,GE65_REDACT_FLAG,TOTAL_DAY_SUPPLY_GE65,TOTAL_DRUG_COST_GE65,hsanum,hsacity,hsastate,hrrnum,hrrcity,hrrstate
16490,1326035650,ABBOTT,FAITH,SAGINAW,MI,Neurology,S,HYDROCODONE-ACETAMINOPHEN,HYDROCODONE/ACETAMINOPHEN,NaN,...,NaN,*,NaN,NaN,23098,Saginaw,MI,NaN,Saginaw,MI
23861,1518077056,ABDEL-HALIM,JAMAL,WELLINGTON,FL,Neurology,S,AVINZA,MORPHINE SULFATE,NaN,...,0,NaN,0,0,17121,Wellington,KS,NaN,Wichita,KS
23878,1518077056,ABDEL-HALIM,JAMAL,WELLINGTON,FL,Neurology,S,HYDROCODONE-ACETAMINOPHEN,HYDROCODONE/ACETAMINOPHEN,25,...,36,NaN,1065,577,17121,Wellington,KS,NaN,Wichita,KS
23887,1518077056,ABDEL-HALIM,JAMAL,WELLINGTON,FL,Neurology,S,MORPHINE SULFATE,MORPHINE SULFATE,NaN,...,0,NaN,0,0,17121,Wellington,KS,NaN,Wichita,KS
23888,1518077056,ABDEL-HALIM,JAMAL,WELLINGTON,FL,Neurology,S,MORPHINE SULFATE ER,MORPHINE SULFATE,NaN,...,NaN,*,NaN,NaN,17121,Wellington,KS,NaN,Wichita,KS


In [169]:
subset.to_csv('opioid_data.csv')